# STM Science Assessor
This notebook aims to ingest a table of source/observation specs with their achievability rating per detector model (as generated by prior notebook, the Observation Targets Assessor), and convolve with the Science STM objectives, each of which involves one or more of those source/observation specs. The output should be a similar table to the input, but with the source/observvation specs replaced by objectives.

To run this on Google Colab:
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnGBaker/GWI-metrics/blob/main/Notebooks/STM_Science_Assessor.ipynb)

### Some basic setup

In [1]:
if 'google.colab' in str(get_ipython()):
    !git clone https://github.com/JohnGBaker/GWI-metrics.git
    src='GWI-metrics/src/'
    datadir='GWI-metrics/Data/'
else:
    src='../src/'
    datadir='../Data/'
!mkdir -p '../plots/'

# Import
import os
from datetime import date
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append(src)
import constants
import metrics
import sources
import concepts
import subsystems
import background
import pandas as pd
try:
    import dataframe_image as dfi
except:
    dfi=None
from glob import glob
import re
from astropy.coordinates import Angle
from astropy.cosmology import WMAP9 as cosmo
from astropy import units as u

pd.set_option('display.float_format', '{:.2E}'.format)

LARGENUM = 1.0e10 # This is a hard-coded number to beat
verbose = False

### Define the set of concepts
Here we load in a set of the pre-defined concepts, or you can define your own

In [2]:
missionNames = (
    'LISACBE',
    #'LISASciRDv1',
    'TwinLISA',
    'LISAGrande',
    'LISAU',
    'GoBIGLISA',
    'ALIA',
    'ALIAtwin',
    'GoBIGALIA')
    #'ALIAlowL')
missions=[concepts.menu[mission] for mission in missionNames]

Nmissions = len(missionNames)

for mission in missions:
    mission=background.add2model(mission)
#model = concepts.LISACBE.copy()
#modelName = model.get('label')
#model = background.add2model(model)     # add galactic background model

### Load the observation target data
Load information about the target sources and the observation quality targets from the STM.
The data are from https://nasa.sharepoint.com/:x:/t/GravitationalWaveImager/EaaeMC7L-2NJpFCQloYbSnoBDZag_cvFWR5_BjRoFAD6Tw?e=uWr4mU

In [3]:
# Read in the most recent spec file produced by other Notebook:
files=glob(datadir+'ScoredObservations_net_*.csv')
#files.sort(key=os.path.getmtime)
files=sorted(files)
specfile=files[-1]
print('Most recent specs file is:',specfile)

# Are objective names actually stored?
# A: not originally -- had to re-run earlier notbook with the "index=None" removed from the CVS save command

speclist = []
with open(specfile) as f:
    for row in f:
        speclist.append(row.split(",")[0])

speclist.pop(0) # remove the top item, which is empty "corner" element of table
print("specs are: ",speclist)

Nspecs = len(speclist)

#specarray = np.loadtxt(specfile,delimiter=',',skiprows=1,usecols = range(1,Nmissions+1))

rawspecarray = np.genfromtxt(specfile,delimiter=',')

specarray=rawspecarray[1:Nspecs+1,1:Nmissions+1]



    

Most recent specs file is: ../Data/ScoredObservations_net_2022_09_26.csv
specs are:  ['XX1.1.bXX', '1.1.c', '1.1.d', '1.1.e', '1.1.f', '1.1.g', '1.1.h', '1.1.i', 'XX1.2.bXX', '1.2.c', '1.2.d', '1.2.e', 'XX1.3.bXX', '1.3.c', '1.3.d', '1.3.e', '1.3.f', '1.3.g', '1.4.a', '2.1.a', '2.1.b', '2.1.c', '2.1.d', '2.1.e', '2.1.f', '2.1.g', 'XX2.2.bXX', '2.3.b', '2.3.c', '2.4.a', '2.4.b', '2.4.c', 'XX3.1.aXX', '3.1.b', 'XX3.2.aXX', '3.2.b', '3.3.a', '3.3.b', '3.3.c', 'XX3.3.dXX', '3.3.e', '3.3.f', '3.4.a', '3.4.b', '3.5.a', '3.6.a', '3.7.a', 'XX4.1.aXX', 'XX4.1.bXX', '4.1.c', '4.1.d', 'XX5.1.aXX', 'XX5.1.bXX', 'XX5.1.cXX', 'XX5.1.dXX', '5.1.e', 'XX5.2.aXX', 'XX5.2.bXX', '5.2.d', 'XX5.3.aXX', 'XX5.3.bXX', '5.3.d', 'XX5.4.aXX', 'XX5.4.bXX', '5.4.d', '5.5.a', 'XX5.5.bXX', '5.5.c', '5.5.d', '5.6.a', '5.6.b', 'XX5.7.aXX', '5.7.b', '5.8.a']


In [4]:
STMfiles=glob(datadir+'/ScienceSTM*.csv')
STMfiles=sorted(STMfiles)
#STMfiles.sort(key=os.path.getmtime)
file=STMfiles[-1]
#file=glob('../Data/ScienceSTM*.csv')[0]
print('Science STM file is:',file)

#Read file
df=pd.read_csv(file,header=1)
print(len(df),'rows read from file.')

if True:
    #Drop empty rows
    for i,row in df.iterrows():
        if row.isnull().all():
            df=df.drop(i)
    #Fill empties in first column with values above

    nrows = len(df. index) 

    newdata = np.zeros((nrows,Nmissions))

    val=float('nan')
    
    col='Astrophysical Parameters (Level 1 Measurement Req)'
    requirementNames=df[col].values
    if verbose:
        print("requirement Names are ",requirementNames)

    analysisVal=float('nan')
    analysisCol='Specific Analyses'
    analysisNames=df[analysisCol].values

    if verbose:
        print("analyses are: ",analysisNames)
    
    analysisCodes = analysisNames
    for j in range(len(analysisNames)):
        analysisName_here = analysisNames[j]
        #print("this analysis names is ",analysisName_here)
        #found = re.search("\w\w-\w\w",analysisName_here).group(1)
        found = re.findall("\w\w-\w\w",analysisName_here)
        #print("analysis Name ",analysisName_here,' has code ',found[0])
        analysisCodes[j] = found[0]
        
    actionVal=float('nan')
    actionCol='Science Actions'
    actionNames=df[actionCol].values

    cleanedList = [x for x in actionNames if str(x) != 'nan']
    actionNames = np.array(cleanedList)
    actionCodes = actionNames
    Nactions = len(actionNames)
    for j in range(Nactions):
        actionName_here = actionNames[j]
        found = re.findall("\w\w-\w",actionName_here)
        actionCodes[j] = found[0]
    if verbose:
        print("actions are: ",actionNames)        
        print('action codes are: ',actionCodes)
    
    useAnalysis = np.full(len(analysisCodes),False)
    
    df['obslist']=None
    allusedspecs=[]
    
    for i,row in df.iterrows(): #each row should be a specific analysis in analysisNames
        if row.isnull()[col]:
            val = ''
            #df.loc[i,col]=val
        else:
            val=df.loc[i,col]

        if row.isnull()[analysisCol]:
            df.loc[i,analysisCol]=analysisVal
        else:
            analysisVal=df.loc[i,analysisCol]    
            
        bestspecres_here = LARGENUM*np.ones(Nmissions)

        # parse to get values in form of 2.3.b == <single digit> + dot + <single digit> + dot + <single letter>
        # or perhaps just a comma-separated list that can be compared with other file
        speccases_here = val.split(",")

        Nspeccases_here = len(speccases_here)
        Nbadspecs = 0 # counting bad/missing specs for this analysis
        specs=[]
        for ispeccases_here in range(Nspeccases_here): # loop over all measure requirements associated with this Specific Analysis
            speccase_here = (speccases_here[ispeccases_here]).strip() # remove leading whitespace
            # look for match with list of specs read in earlier
            # if it's present, combine with existing data for this requirement -- how?
            # for now: try smallest number
            specres_here = 2.0*LARGENUM*np.ones(Nmissions)
            try:
                specidx = speclist.index(speccase_here)
                specs.append(speclist[specidx])
                specres_here = specarray[specidx,:]
            except ValueError:
                print("specification ",speccase_here," not generated by earlier Notebook! Ignoring")
                Nbadspecs = Nbadspecs + 1
           
            #bestspecres_here = np.minimum(bestspecres_here,specres_here)
            bestspecres_here = np.fmin(bestspecres_here,specres_here) # get minimum over all non-NaN values

        #print("For this requirement, the best spec results for each detector are ",bestspecres_here)
        if Nbadspecs<Nspeccases_here:
            newdata[i,:] = bestspecres_here
            useAnalysis[i] = True
            
        # This is a Ndetectors-length array for this particular Specific Analysis
        
        #Store results about which observations specifications were used
        analysisCodes[i]+=': '+str(specs)
        allusedspecs+=specs
        
    # Want to cut the main analysis list down using the useAnalysis array
    goodAnalysisCodes=analysisCodes[useAnalysis]
    goodAnalysisData=newdata[useAnalysis,:]
    
    allusedspecs=list(set(allusedspecs))
    unusedspecs=[x for x in speclist if not x in allusedspecs]
    print('Unused observation targets:',unusedspecs)


Science STM file is: ../Data/ScienceSTM_2022-09-26.csv
75 rows read from file.
specification  FIXME  not generated by earlier Notebook! Ignoring
specification  FIXME  not generated by earlier Notebook! Ignoring
specification  FIXME  not generated by earlier Notebook! Ignoring
specification  4.1.e  not generated by earlier Notebook! Ignoring
specification  FIXME  not generated by earlier Notebook! Ignoring
specification  FIXME  not generated by earlier Notebook! Ignoring
specification  FIXME  not generated by earlier Notebook! Ignoring
specification  FIXME  not generated by earlier Notebook! Ignoring
specification  FIXME  not generated by earlier Notebook! Ignoring
specification  FIXME  not generated by earlier Notebook! Ignoring
specification  FIXME  not generated by earlier Notebook! Ignoring
specification  FIXME  not generated by earlier Notebook! Ignoring
specification  FIXME  not generated by earlier Notebook! Ignoring
specification  FIXME  not generated by earlier Notebook! Ignori

### Perform the observations and assessment
We first compute the SNR and then the angular resolution

In [5]:
#pd.set_option('display.float_format', '{:.2g}'.format)
#adfnew = pd.DataFrame(newdata,index=analysisCodes,columns=missionNames)
adfnew = pd.DataFrame(goodAnalysisData,index=goodAnalysisCodes,columns=list(missionNames))
#print(adfnew)

#adf=adf.dropna()
def shrink():
    return [dict(selector="th",
                 props=[("font-size", "6pt")]),
            dict(selector="td",
                 props=[('padding', "0em 0em")])
]
def zoom():
    return [
            dict(selector="th:hover",
                 props=[("font-size", "12pt")]),
            dict(selector="tr:hover td:hover",
                 props=[('max-width', '200px'),
                        ('font-size', '12pt')])
]

#pd.set_option('display.float_format', '{:.2g}'.format)
s=adfnew.style.format('{:.2E}').background_gradient(vmin=0.1,vmax=10,cmap='RdYlGn_r').set_properties(**{'max-width': '120px', 'font-size': '6pt'})
#s=s.set_table_styles(shrink()+zoom())
s=s.set_caption("Net performance")
display(s)

saveSpecificAnalysesTable=True
dates = date.today().strftime("%Y_%m_%d")
if saveSpecificAnalysesTable and dfi is not None:
    outimagename = datadir+'ScoredRequirements_net_'+dates+'.png'
    dfi.export(
        s,
        outimagename
    )

if True:
    outfile=datadir+'ScoredRequirements_net_'+dates+'.csv'
    #adfnew.to_csv(outfile,index=False)
    adfnew.to_csv(outfile,index=True)
    print('Wrote to file:',outfile)


,LISACBE,TwinLISA,LISAGrande,LISAU,GoBIGLISA,ALIA,ALIAtwin,GoBIGALIA
"1a-1a: ['2.1.b', '2.3.b']",1.78E+03,2.76E+01,2.33E+01,2.37E-01,4.59E-01,2.69E+03,2.76E+01,9.19E-01
1a-1b: ['1.1.f'],3.43E+02,2.42E+02,4.00E+02,1.31E+03,3.57E+02,1.99E+01,1.41E+01,1.41E+01
1a-1c: ['1.2.c'],6.93E+01,4.90E+01,8.04E+01,2.64E+02,7.20E+01,4.35E+00,3.07E+00,3.07E+00
1a-1e: ['1.4.a'],5.80E+01,3.99E+01,6.76E+01,2.17E+02,5.90E+01,3.03E+00,2.14E+00,2.14E+00
"1a-2a: ['2.1.f', '2.3.c']",1.81E+02,2.14E+00,3.23E+00,4.48E-01,5.21E-02,3.89E+02,9.20E-01,3.07E-02
1a-2b: ['1.3.f'],4.48E+03,2.57E+03,3.83E+03,7.94E+03,1.80E+02,6.47E+02,3.27E+02,1.56E+01
1a-2c: ['1.1.g'],1.40E+05,8.08E+04,1.61E+05,3.37E+05,6.88E+03,8.30E+03,4.78E+03,2.74E+02
1a-2d: ['1.2.d'],5.25E+04,3.03E+04,5.97E+04,1.24E+05,2.55E+03,3.60E+03,2.05E+03,1.16E+02
"1a-2f: ['3.1.b', '3.2.b', '3.3.f', '3.4.b']",4.03E-01,1.88E-01,4.78E-02,3.91E-02,6.58E-02,3.70E-01,1.51E-01,8.84E-02
"1a-2g: ['5.1.e', '5.2.d', '5.3.d', '5.4.d', '5.5.d', '5.6.b', '5.7.b']",5.17E+02,1.64E+02,8.99E+01,1.68E+01,4.34E+00,2.11E+02,2.42E+01,8.09E-01


[0926/135930.660452:INFO:headless_shell.cc(660)] Written to file /var/folders/x6/0_5snvmj5rj3s1z43y5f3vncf2_rs9/T/tmpnn9fe828/temp.png.
[0926/135932.162451:INFO:headless_shell.cc(660)] Written to file /var/folders/x6/0_5snvmj5rj3s1z43y5f3vncf2_rs9/T/tmpc4z1g18g/temp.png.


Wrote to file: ../Data/ScoredRequirements_net_2022_09_26.csv


In [6]:
actionData = np.zeros((Nactions,Nmissions))

#print("all analysis codes are: ",goodAnalysisCodes)

Nanalyses = len(goodAnalysisCodes)

useAction = np.full(Nactions,False)

for iact in range(Nactions):
    thisActionCode = actionCodes[iact]
    actionMatchList = np.full(Nanalyses,False)
    for janal in range(Nanalyses):
        actionMatchList[janal]=thisActionCode in goodAnalysisCodes[janal]
    #thisActionAnalyses = thisActionCode in goodAnalysisCodes
    #print(thisActionAnalyses)
    if any(actionMatchList):
        #print("action code",thisActionCode," is in these analysis codes:", goodAnalysisCodes[actionMatchList])
    
        thisActionDataContribs = goodAnalysisData[actionMatchList,:]
        #print(thisActionDataContribs)
        thisActionData=np.amin(thisActionDataContribs,axis=0)
        #print(thisActionData)
        useAction[iact] = True
        actionData[iact,:]=thisActionData
        
#print("useAction array is ",useAction)
#print("full set of action codes is ",actionCodes)
        
goodActionCodes=actionCodes[useAction]
goodActionData=actionData[useAction,:]

adfnew = pd.DataFrame(goodActionData,index=goodActionCodes,columns=list(missionNames))
#print(adfnew)

#adfnew.style.format('{:.2E}')
#adfnew.style.background_gradient(vmin=0.1,vmax=10,cmap='RdYlGn_r').set_properties(**{'max-width': '120px', 'font-size': '6pt'})
#display(adfnew)

#s=adfnew.style.background_gradient(vmin=0.1,vmax=10,cmap='RdYlGn_r').set_properties(**{'max-width': '120px', 'font-size': '6pt'})
s=adfnew.style.format('{:.2E}').background_gradient(vmin=1.0E-1,vmax=1.0E1,cmap='RdYlGn_r').set_properties(**{'max-width': '120px', 'font-size': '6pt'})
s=s.set_caption("Net performance by actions")
display(s)


saveSpecificActionsTable=True
dates = date.today().strftime("%Y_%m_%d")
if saveSpecificActionsTable and dfi is not None:
    outimagename = datadir+'ScoredActions_net_'+dates+'.png'
    dfi.export(
        s,
        outimagename
    )

if True:
    outfile=datadir+'ScoredActions_net_'+dates+'.csv'
    #adfnew.to_csv(outfile,index=False)
    adfnew.to_csv(outfile,index=True)
    print('Wrote to file:',outfile)


,LISACBE,TwinLISA,LISAGrande,LISAU,GoBIGLISA,ALIA,ALIAtwin,GoBIGALIA
1a-1,5.80E+01,2.76E+01,2.33E+01,2.37E-01,4.59E-01,3.03E+00,2.14E+00,9.19E-01
1a-2,4.03E-01,1.88E-01,4.78E-02,3.91E-02,5.21E-02,3.70E-01,1.51E-01,3.07E-02
1a-3,7.47E+01,4.28E+01,6.38E+01,1.32E+02,5.06E+00,1.08E+01,5.46E+00,7.88E-01
1a-4,1.15E+03,8.12E+02,1.33E+03,4.35E+03,1.19E+03,7.55E+01,5.34E+01,5.34E+01
1b-1,7.49E+00,8.85E-02,1.08E-01,1.49E-02,1.77E-03,2.16E+01,7.16E-02,2.39E-03
1b-3,8.79E+03,5.06E+03,8.60E+03,1.79E+04,3.85E+02,1.04E+03,5.52E+02,2.77E+01
1d-1,1.45E+03,1.72E+01,4.38E+01,6.41E+00,4.61E-01,2.18E+03,5.15E+00,1.72E-01
1d-2,1.77E+03,1.02E+03,1.80E+02,6.42E+01,2.99E+01,1.76E+03,1.02E+03,5.85E+01
1d-3,8.07E+03,9.53E+01,1.11E+02,1.50E+01,1.87E+00,3.36E+04,7.94E+01,2.65E+00
2a-1,7.49E+00,8.85E-02,1.08E-01,1.49E-02,1.77E-03,3.03E+01,7.16E-02,2.39E-03


[0926/135933.746288:INFO:headless_shell.cc(660)] Written to file /var/folders/x6/0_5snvmj5rj3s1z43y5f3vncf2_rs9/T/tmpv8a43rlh/temp.png.


Wrote to file: ../Data/ScoredActions_net_2022_09_26.csv
